In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from inspect import getfullargspec
from importlib import import_module
import sys
from pathlib import Path
from distutils.util import strtobool



In [40]:
import ArgConfigParse
from epdlib import Screen
from epdlib.Screen import ScreenError

In [4]:
from library.Plugin import Plugin
from library.CacheFiles import CacheFiles
import my_constants as constants

In [5]:
logger = logging.getLogger(__name__)

In [6]:
def do_exit(status=0, message=None, **kwargs):
    '''exit with optional message
    Args:
        status(int): integers > 0 exit with optional message
        message(str): optional message to print'''
    if message:
        if status > 0:
            logger.error(f'failure caused exit: {message}')
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [7]:
def config_str_to_val(config):
    '''convert strings in config dictionary into appropriate types
             float like strings ('7.1', '100.2', '-1.3') -> to float
             int like strings ('1', '100', -12) -> int
             boolean like strings (yes, no, Y, t, f, on, off) -> 0 or 1
             
         Args:
             config(`dict`): nested config.ini style dictionary

         Returns:
             `dict`'''    

    def eval_expression(string):
        '''safely evaluate strings allowing only specific names
        see: https://realpython.com/python-eval-function/
        
        e.g. "2**3" -> 8; "True" -> True; '-10.23' -> 10.23
        
        Args:
            string(str): string to attempt to evaluate
            
        Returns:
            evaluated as bool, int, real, etc.'''
        
        # set dict of allowed strings to and related names e.g. {"len": len}
        allowed_names = {}
        
        # compile the string into bytecode
        code = compile(string, "<string>", "eval")
        
        # check .co_names on the bytecode object to make sure it only contains allowed names
        for name in code.co_names:
            if name not in allowed_names:
                # raise a NameError for any name that's not allowed
                raise NameError(f'use of {name} not allowed')
        return eval(code, {"__builtins__": {}}, allowed_names)
    
    def convert(d, function, exceptions):
        '''convert value strings in dictionary to appropriate type using `function`
        
        d(dict): dictionary of dictionary of key/value pairs
        function(`func`): type to convert d into
        exceptions(tuple of Exceptions): tuple of exception types to ignore'''
        for section, values in d.items():
            for key, value in values.items():
                if isinstance(value, str):
                    try:
                        sanitized = function(value)
                    except exceptions:
                        sanitized = value

                    d[section][key] = sanitized
                else:
                    d[section][key] = value
        return d
    
    # evaluate int, float, basic math: 2+2, 2**15, 23.2 - 19
    convert(config, eval_expression, (NameError, SyntaxError))
    # convert remaining strings into booleans (if possible)
    # use the distuitls strtobool function
    convert(config, strtobool, (ValueError, AttributeError))
    
    # return converted values and original strings
    
    return config

In [8]:
def get_cmd_line_args():
    '''process command line arguments
    
    Returns:
        dict of parsed config values'''
    
    cmd_args = ArgConfigParse.CmdArgs()
    
    cmd_args.add_argument('-c', '--config', ignore_none=True, metavar='CONFIG_FILE.ini',
                         type=str, dest='user_config',
                         help='use the specified configuration file')
    
    cmd_args.add_argument('-l', '--log_level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
                         dest='main__log_level', help='change the log output level')
    
    cmd_args.add_argument('-d', '--daemon', required=False, default=False,
                         dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration if found)')    
    
    cmd_args.parse_args()    

    return cmd_args

In [9]:
def get_config_files(cmd_args):
    '''read config.ini style files(s)
    
    Args: 
        cmd_args(`ArgConfigParse.CmdArgs` obj)
        
    Returns:
        ArgConfigParse.ConfgifFile'''
    
    logger.debug('gathering configuration files')
    
    config_files_dict = {'base': constants.CONFIG_BASE,
                         'system': constants.CONFIG_SYSTEM,
                         'user': constants.CONFIG_USER,
                         'cmd_line': cmd_args.options.user_config}
    
    config_files_list = [config_files_dict['base']]
    
    if cmd_args.options.main__daemon:
        logging.debug(f'using daemon configuration: {constants.CONFIG_SYSTEM}')
        config_files_list.append(config_files_dict['system'])
    else:
        if constants.CONFIG_USER.exists():
            config_files_list.append(config_files_dict['user'])
        else:
            try:
                constants.CONFIG_USER.parent.mkdir(parents=True, exist_ok=True)
            except PermissionError as e:
                msg=f'could not create user configuration directory: {constants.CONFIG_USER.parent}'
                logger.critical(msg)
                do_exit(1, msg)
            try:
                shutil.copy(constants.CONFIG_BASE, constants.CONFIG_USER)
            except Exception as e:
                msg=f'could not copy user configuration file to {constants.CONFIG_USER}'
                logging.critical(1, msg)
                do_exit(1, msg)
            msg = f'''This appears to be the first time PaperPi has been run.
A user configuration file created: {constants.CONFIG_USER}
At minimum you edit this file and add a display_type and enable one plugin.
        
Edit the configuration file with:
   $ nano {constants.CONFIG_USER}'''
            do_exit(0, msg)
    
    logger.info(f'using configuration files to configure PaperPi: {config_files_list}')
    config_files = ArgConfigParse.ConfigFile(config_files_list, ignore_missing=True)
    try:
        config_files.parse_config()
    except DuplicateSectionError as e:
        logger.error(f'{e}')
        config_files = None

    return config_files


In [10]:
def build_plugins_list(config, resolution, cache):
    '''Build a dictionary of configured plugin objects
    
    Args:
        config(dict): configuration dictionary 
        resolution(tuple): X, Y resolution of screen
        cache(obj: Cache): cache object for managing downloads of images
        
    Returns:
        dict of Plugin'''
    
    def font_path(layout):
        '''add font path to layout'''
        for k, block in layout.items():
            font = block.get('font', None)
            if font:
                font = font.format(constants.FONTS)
                block['font'] = font
        return layout
    
    # get the expected key-word args from the Plugin() spec
    spec_kwargs = getfullargspec(Plugin).args

    plugins = []
    
    config_dict = config_str_to_val(config.config_dict)

    
    for section, values in config_dict.items():
        if section.startswith('Plugin:'):
            logger.info(f'[[ {section} ]]')
            
            plugin_config = {}
            # add all the spec_kwargs from the config
            plugin_kwargs = {}
            for key, val in values.items():
                if key in spec_kwargs:
                    plugin_config[key] = val
                else:
                    # add everything that is not one of the spec_kwargs to this dict
                    plugin_kwargs[key] = val

            # populate the kwargs plugin_config dict that will be passed to the Plugin() object
            plugin_config['name'] = section
            plugin_config['resolution'] = resolution
            plugin_config['config'] = plugin_kwargs
            plugin_config['cache'] = cache
            # force layout to one-bit mode for non-HD screens
            if not config_dict['main'].get('display_type') == 'HD':
                plugin_config['force_onebit'] = True
            try:
                module = import_module(f'{constants.PLUGINS}.{values["plugin"]}')
                plugin_config['update_function'] = module.update_function
                layout = getattr(module.layout, values['layout'])
                layout = font_path(layout)
                plugin_config['layout'] = layout
            except KeyError as e:
                logger.info('no module specified; skipping update_function and layout')
                continue
            except ModuleNotFoundError as e:
                logger.warning(f'error: {e} while loading module {constants.PLUGINS}.{values["plugin"]}')
                logger.warning(f'skipping plugin')
                continue
            except AttributeError as e:
                logger.warning(f'could not find layout "{plugin_config["layout"]}" in {plugin_config["name"]}')
                logger.warning(f'skipping plugin')
                continue
            my_plugin = Plugin(**plugin_config)
            try:
                my_plugin.update()
            except AttributeError as e:
                logger.warning(f'ignoring plugin {my_plugin.name} due to missing update_function')
                logger.warning(f'plugin threw error: {e}')
                continue    
            logger.info(f'appending plugin {my_plugin.name}')
            
            
            plugins.append(my_plugin)
            
    
    return plugins

In [22]:
def setup_splash(config, resolution):
    if config['main'].get('splash', False):
        logger.debug('displaying splash screen')
        from plugins.splash_screen import splash_screen
        splash_config = {
            'name': 'Splash Screen',
            'layout': splash_screen.layout.layout,
            'update_function': splash_screen.update_function,
            'resolution': resolution
        }
        splash = Plugin(**splash_config)
        splash.update(constants.APP_NAME, constants.VERSION, constants.URL)
    else:
        logger.debug('skipping splash screen')
        splash = False
    
    return splash

In [37]:
def setup_display(config):
    def ret_obj(obj=None, status=0, message=None):
        return{'obj': obj, 'status': status, 'message': message} 
    
    keyError_fmt = 'configuration KeyError: section[{}], key: {}'    
    
    moduleNotFoundError_fmt = 'could not load epd module {} -- error: {}'
    
    epd = config['main'].get('display_type', None)
    vcom = config['main'].get('vcom', None)
    
    try:
        screen = Screen(epd=epd, vcom=vcom)
        screen.clearEPD()
    except ScreenError as e:
        logging.critical('Error loading epd from configuration')
        return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(epd, e))
        return return_val
    except PermissionError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'The user executing {constants.app_name} does not have access to the SPI device.')
        return_val = ret_obj(None, 1, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')
        return return_val
    except FileNotFoundError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'It appears that SPI is not enabled on this Pi. See: https://github.com/txoof/epd_display/tree/testing#hardwareos-setup')
        return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(epd, e))
        return return_val
    
    
    try:
        screen.rotation = config['main'].get('rotation', 0)
    except ValueError as e:
        logger.error('invalid screen rotation [main][rotation] - acceptable values are (0, 90, -90, 180)')
        return_val = ret_obj(None, 1, keyError_format.format('main', 'rotation'))
        return return_val
    
    return ret_obj(obj=screen)    

In [43]:
ms = setup_display(c.config_dict)

DEBUG:root:configuring IT8951 epd
DEBUG:root:epd configuration {'epd': <IT8951.display.AutoEPDDisplay object at 0xacc43910>, 'resolution': [1200, 825], 'clear_args': {}, 'one_bit_display': False, 'constants': <module 'IT8951.constants' from '/home/pi/.local/share/virtualenvs/paperpi-fN25Eeb-/lib/python3.9/site-packages/IT8951/constants.py'>, 'mode': 'L'}
DEBUG:root:rotation=0, resolution=[1200, 825]
DEBUG:root:initing display
DEBUG:root:HD display
DEBUG:root:clearing screen
DEBUG:root:sleeping display
DEBUG:root:rotation=0, resolution=[1200, 825]


In [11]:
a = get_cmd_line_args()
c = get_config_files(a)
my_cache = CacheFiles()

DEBUG:ArgConfigParse.ArgConfigParse:options: Namespace(user_config=None, main__log_level=None, main__daemon=False)
DEBUG:ArgConfigParse.ArgConfigParse:ignoring key: user_config
DEBUG:ArgConfigParse.ArgConfigParse:ignoring key: main__log_level
INFO:ArgConfigParse.ArgConfigParse:ignoring unknown options: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-09f340ec-500e-4e2c-ab30-3f06435b4d0d.json']
DEBUG:__main__:gathering configuration files
INFO:__main__:using configuration files to configure PaperPi: [PosixPath('config/paperpi.ini'), PosixPath('/home/pi/.config/com.txoof.paperpi/paperpi.ini')]
DEBUG:ArgConfigParse.ArgConfigParse:received config_file list: [PosixPath('config/paperpi.ini'), PosixPath('/home/pi/.config/com.txoof.paperpi/paperpi.ini')]
DEBUG:ArgConfigParse.ArgConfigParse:/home/pi/src/paperpi/paperpi/config/paperpi.ini exists
DEBUG:ArgConfigParse.ArgConfigParse:/home/pi/.config/com.txoof.paperpi/paperpi.ini exists
DEBUG:ArgConfigParse.ArgConfigParse:processing existing co

In [14]:
pl = build_plugins_list(c, (640, 480), my_cache)

INFO:__main__:[[ Plugin: default fallback plugin ]]
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [----------digit_time----------]
DEBUG:root:section: [-------------msg--------------]
DEBUG:root:[[....calculating layouts....]]
INFO:root:section: [..........digit_time..........]
DEBUG:root:resolution: (640, 480)
DEBUG:root:width: 1, height: 0.5
DEBUG:root:scaling font size
DEBUG:root:y target size reached
DEBUG:root:calculated font size: 60
DEBUG:root:absolute coordinates provided
DEBUG:root:block coordinates: (0, 0)
INFO:root:section: [.............msg..............]
DEBUG:root:resolution: (640, 480)
DEBUG:root:width: 1, height: 0.5
DEBUG:root:scaling font size
DEBUG:root:y target size reached
DEBUG:root:calculated font size: 44
DEBUG:root:absolute coordinates provided
DEBUG:root:block coordinates: (0, 240)
INFO:root:[[____setting blocks____]]
INFO:root:section: [__________digit_time__________]
DEBUG:root:setting block type: TextBlock
DEBUG:root:block are

In [13]:
logging.basicConfig(level='INFO')